### 8.
In some cases, it is necessary to efficiently perform insertion or removal of elements 
both at the beginning and at the end of the collection. Measure the time for the 
following operations with N = 10 000, 20 000 y 30 000 elements: 
a. Delete last element of a list via pop() 
b. Delete first element of a list via pop(0) 
c. Append 1 at the end of the list. 
d. Insert 1 at the beginning of the list insert(0, 1) 
Make a table with your results. It should looks like table on Chapter 2: Pure Python 
Optimization (pp. 38) from the book G. Lenaro (2017). Python high Performance. 
Second Edition. UK: Packt Publishing Ltd. 

In [1]:
import pandas as pd
import timeit

In [2]:
size = [10_000, 20_000, 30_000]
operations = {
    "list.pop()" : 'lst.pop()',
    "list.pop(0)" : 'lst.pop(0)',
    "list.append(1)" : 'lst.append(1)',
    "list.insert(0, 1)" : 'lst.insert(0, 1)',
}

results = {op: [] for op in operations}

for _ in size:
    for operation, function in operations.items():
        lst_code = f"lst = list(range({_}))"
        time = timeit.timeit(stmt=function, setup=lst_code, number=1000)
        time /= 1000 * 1e6  

        results[operation].append(time)


df = pd.DataFrame(results, index=[n for n in size])

df_transposed = df.transpose()

df_transposed.head()

,10000,20000,30000
list.pop(),2.314700e-14,2.319500e-14,2.311400e-14
list.pop(0),1.251086e-12,2.552214e-12,4.559416e-12
list.append(1),1.718000e-14,2.631100e-14,2.068700e-14
"list.insert(0, 1)",2.778298e-12,4.277503e-12,7.327114e-12


### 9.
Python provides a data structure with interesting properties in the collection.deque 
class. The word deque stands for double-ended queue because this data structure 
is designed to efficiently put and remove elements at the beginning and at the end 
of the collection. Evaluate the following methods with N = 10 000, 20 000 and 30 
000 elements: 
a. deque.pop() 
b. deque.popleft() 
c. deque.append(1) 
d. deque.appendleft(1) 
Make a table with your results. It should looks like table on pp. 39 on the same book 
as previous task.  

In [3]:
operations = {
    'deque.pop()': 'dq.pop()',
    'deque.popleft': 'dq.popleft()',
    'deque.append(1)': 'dq.append(1)',
    'deque.appendleft(1)': 'dq.appendleft(1)'
}

results = {op: [] for op in operations}

for n in size:
    for operation, function in operations.items():
        dqncode = f"from collections import deque\n" \
                   f"dq = deque(range({n}))"
        time = timeit.timeit(stmt=function, setup=dqncode, number=1000) / 1000 * 1e6
        results[operation].append(time)

df = pd.DataFrame(results, index=[n for n in size])

df_transposed = df.transpose()

df_transposed

,10000,20000,30000
deque.pop(),0.022583,0.021570,0.022064
deque.popleft,0.027129,0.020038,0.020417
deque.append(1),0.020093,0.019832,0.039632
deque.appendleft(1),0.021218,0.021368,0.029661


### 10.
 The efficiency gained by the appendleft and popleft comes at a cost: accesing an 
element in the middle of a deque is a O(N) operation. Evaluate the time for the next 
operations with with N = 10 000, 20 000 and 30 000 elements: 
a. deque[0] 
b. deque[N-1] 
c. deque[int(N/2)] 
Make a table with your results. It should looks like table on pp. 39 on the same book 
as previous task.

In [4]:
operations = {
    'deque[0]': 'dq[0]',
    'deque[N-1]': 'dq[N-1]',
    'deque[int(N/2)]': 'dq[int(N/2)]',
}
results = {op: [] for op in operations}

for n in size:
    for op, stmt in operations.items():
        dq_code = f'from collections import deque; N = {n}; dq = deque(range(N))'
        time = timeit.timeit(stmt=stmt, setup=dq_code, number=1000) / 1000 * 1e6
        results[op].append(time)

df_results = pd.DataFrame(results, index=[n  for n in size])

df_transposed = df_results.T

df_transposed

,10000,20000,30000
deque[0],0.019881,0.018918,0.018593
deque[N-1],0.029371,0.039377,0.028649
deque[int(N/2)],0.161288,0.242126,0.439874


### 11.
Explain what is Overallocation in lists.  


Overallocation in Python lists refers to the strategy where Python pre-allocates more memory than is currently needed for a list's elements. This is done to optimize append operations by minimizing the number of times the list needs to reallocate memory as it grows.

When you append to a list in Python, it doesn't necessarily allocate space for just one additional element. Instead, it allocates room for a number of elements in advance. The exact algorithm for this growth strategy is an implementation detail and may vary, but the idea is to balance memory usage with the need for speed of append operations.

Looking at the list operations you provided, list.append(1) has consistently low times regardless of the list size. This demonstrates overallocation at work: most of the time, appending doesn't require a memory reallocation because there's already space reserved for new elements.

In contrast, operations like list.insert(0, 1) show a linear increase in time as the list size grows, because inserting at the beginning of a list forces all the subsequent elements to move one space in memory, which is costly. This operation does not benefit from overallocation because it doesn’t involve simply adding an element at the end but rather reorganizing the existing ones.

The overallocation mechanism makes appending to lists in Python a very efficient operation (O(1) amortized time complexity), which is why appending to lists can be done quickly even as the list grows in size. However, this also means that a list might use more memory than the exact number of elements it contains at any given time.